In [2]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.1 MB/s eta 0:00:00


In [3]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [4]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [5]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [6]:
env.observation_space.n

16

In [7]:
env.action_space.n

4

In [8]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [9]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 13)

In [10]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 19)

In [11]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 6)

In [12]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 3)

In [13]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 14)

In [14]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [15]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 59/1000
Average number of steps: 12.101694915254237


In [17]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 115/1000
Average number of steps: 14.652173913043478


In [18]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 780/1000
Average number of steps: 43.666666666666664


In [19]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [20]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [22]:
np.all(v_values_1 >= v_values_0)

True

In [23]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [24]:
np.all(v_values_2 >= v_values_1)

True

In [25]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [26]:
np.all(v_values_3 >= v_values_2)

True

In [27]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [28]:
def policy_iteration(env, max_iters=500 ,gamma=0.9):

    # Khởi tạo policy
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    for _ in range(max_iters):
        # Policy Evaluation
        V = policy_evaluation(env, policy)

        policy_stable = True
        for s in range(env.observation_space.n):
            old_action = np.argmax(policy[s])
            q_values = []

            for a in range(env.action_space.n):
                q_value = 0

                for prob, next_state, reward, done in env.P[s][a]:
                    q_value += prob * (reward + gamma * V[next_state])
                q_values.append(q_value)

            best_action = np.argmax(q_values)
            policy[s] = best_action

            if old_action != best_action:
                policy_stable = False

        if policy_stable:
            break

    return V, policy

In [29]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [30]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [31]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [32]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [33]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [34]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 81)

In [35]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 780/1000
Average number of steps: 44.34615384615385


II. Cài đặt Policy_iteration

a. Frozen-lakeV1

In [36]:
Best_Value , Best_Policy = policy_iteration(env)

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged at 80-th iteration.
Converged a

In [37]:
Best_Policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [38]:
play_multiple_times(env, Best_Policy, 1000)

Number of successes: 786/1000
Average number of steps: 42.76972010178117


b. FrozenLake8x8-v1

In [39]:
env = gym.make('FrozenLake8x8-v1', render_mode="ansi")

In [40]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 117-th iteration.


In [41]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [42]:
Best_Value , Best_Policy = policy_iteration(env)

Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 117-th iteration.
Converged at 11

In [43]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 734/1000
Average number of steps: 74.35422343324251


In [44]:
play_multiple_times(env, Best_Policy, 1000)

Number of successes: 759/1000
Average number of steps: 74.32411067193677


c. Taxi-v3

In [45]:
env = gym.make('Taxi-v3', render_mode="ansi")

In [46]:
Best_Value , Best_Policy = policy_iteration(env)

Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged 

In [47]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 116-th iteration.


In [48]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [49]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.081


In [50]:
play_multiple_times(env, Best_Policy, 1000)

Number of successes: 1000/1000
Average number of steps: 12.913


II. Nhận xét
1.  Policy Iteration thường hội tụ nhanh hơn so với Value Iteration vì nó cải thiện chính sách sau mỗi lần đánh giá.

2.  Value Iteration chỉ cần lặp lại một bước đánh giá giá trị cho tất cả các trạng thái, trong khi Policy Iteration cần lặp lại cả hai giai đoạn đánh giá và cải thiện chính sách.

3.  Một ưu điểm của Policy Iteration là nó hội tụ đến chính sách tối ưu nhanh hơn, trong khi Value Iteration chỉ hội tụ đến giá trị tối ưu.

4.  Value Iteration chạy nhanh hơn Policy Iteration trong các toy game

5.  Trong các toy game
  1.  Frozen_lack v1         : Chạy mất 0,548s (value iteration), 14,463 (policy iteration)
  2.  Frozen_lack 8x8_v1     : Chạy mất 5,81s (value iteration) , 66,949 (policy iteration)
  3.  Taxi_v3                : Chạy mất 5,81s (value iteration), 476s (policy iteration)
=> Frozen_lack v1 chạy nhanh nhất